<a href="https://colab.research.google.com/github/EduVic/1806-Algebra/blob/master/Copy_of_Mohr_Circle_Effective_Shear_Stress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import matplotlib.patches as patches
from IPython.display import display

# High-resolution theta array (0 to pi corresponds to 2*theta on Mohr circle)
theta = np.linspace(0, np.pi, 2000)

def get_circle(s1, s3):
    center = (s1 + s3) / 2
    radius = (s1 - s3) / 2
    x = center + radius * np.cos(theta)
    y = radius * np.sin(theta)
    return x, y

# Main plotting function tied to the sliders
def plot_mohr_and_physical(s1, s3, pf, mu):
    # Constrain s1 to be >= s3 to prevent errors
    if s3 > s1:
        s3 = s1 - 0.1

    # Create 1x2 subplots with specific width ratios
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7.5), gridspec_kw={'width_ratios': [1.5, 1]})

    # --- PLOT 1: MOHR CIRCLE ---
    ax1.set_aspect('equal', adjustable='box')

    # Plot Simplified Failure Envelope (Constant Slope)
    sn_range = np.linspace(0, 300, 500)
    tau_envelope = mu * sn_range
    ax1.plot(sn_range, tau_envelope, 'r-', linewidth=2.5, label=rf"Frictional Sliding (Byerlee's Law) ($\mu={mu}$)")

    # Calculate effective stresses
    s1_eff = s1 - pf
    s3_eff = s3 - pf

    # Plot Total Stress Circle
    x_t, y_t = get_circle(s1, s3)
    ax1.plot(x_t, y_t, 'b-', linewidth=2, label='Total Stress')
    ax1.fill(x_t.tolist() + [x_t[-1], x_t[0]],
             y_t.tolist() + [0, 0], color='blue', alpha=0.1)

    # Plot Effective Stress Circle
    x_e, y_e = get_circle(s1_eff, s3_eff)
    ax1.plot(x_e, y_e, 'g-', linewidth=2, label='Effective Stress ($P_f$)')
    ax1.fill(x_e.tolist() + [x_e[-1], x_e[0]],
             y_e.tolist() + [0, 0], color='green', alpha=0.3)

    # --- CALCULATE UNSTABLE ANGLES ---
    tau_b = mu * x_e
    unstable_idx = np.where(y_e >= tau_b)[0]

    # Setup for physical plot
    ax2.set_xlim(-1.2, 1.2)
    ax2.set_ylim(-1.2, 1.2)
    ax2.set_aspect('equal')
    ax2.axis('off')

    # Draw sigma 1 (Horizontal) and sigma 3 (Vertical) arrows
    ax2.annotate('', xy=(-0.4, 0), xytext=(-1.1, 0), arrowprops=dict(facecolor='black', width=3, headwidth=10))
    ax2.annotate('', xy=(0.4, 0), xytext=(1.1, 0), arrowprops=dict(facecolor='black', width=3, headwidth=10))
    ax2.text(-1.2, 0.1, r'$\sigma_1$', fontsize=16, ha='center', va='center')
    ax2.text(1.2, 0.1, r'$\sigma_1$', fontsize=16, ha='center', va='center')

    ax2.annotate('', xy=(0, -0.4), xytext=(0, -1.1), arrowprops=dict(facecolor='black', width=2, headwidth=8))
    ax2.annotate('', xy=(0, 0.4), xytext=(0, 1.1), arrowprops=dict(facecolor='black', width=2, headwidth=8))
    ax2.text(0.15, -1.1, r'$\sigma_3$', fontsize=16, ha='center', va='center')
    ax2.text(0.15, 1.1, r'$\sigma_3$', fontsize=16, ha='center', va='center')

    # Draw rock block outline (circle)
    rock_circle = patches.Circle((0,0), 0.9, color='lightgray', alpha=0.2, zorder=0)
    ax2.add_patch(rock_circle)
    ax2.plot(0.9*np.cos(np.linspace(0, 2*np.pi, 100)), 0.9*np.sin(np.linspace(0, 2*np.pi, 100)), color='gray', linewidth=1)

    # --- LOGIC FIX: Check Hydrofracture FIRST ---
    if pf > s3:
        ax1.set_title(r'HYDROFRACTURE! ($P_f > \sigma_3$)', color='red', fontsize=16, weight='bold')
        ax2.set_title("Tensile Failure\n(Mode I Fracture)", color='red', fontsize=14)
        status_text = "Tensile Failure"
        text_color = 'red'

        # Draw tensile fracture (perpendicular to sigma 3)
        ax2.plot([-0.9, 0.9], [0, 0], 'k-', linewidth=4)

    elif len(unstable_idx) > 0:
        ax1.set_title('Interactive Mohr Circle & Failure Envelope', color='black', fontsize=14)
        ax2.set_title("Physical Orientations\n(Thrust Regime)", color='red', fontsize=14)

        # Highlight the unstable arc in red on Mohr Circle
        ax1.plot(x_e[unstable_idx], y_e[unstable_idx], 'r-', linewidth=4, label='Unstable Orientations')

        # In Mohr space, theta_array is 2*theta. Divide by 2 to get angle between normal and sigma_1
        theta_fault_min = np.degrees(theta[unstable_idx[0]] / 2)
        theta_fault_max = np.degrees(theta[unstable_idx[-1]] / 2)

        # In physical space (sigma_1 horiz), dip angle is 90 - theta
        dip_min = 90 - theta_fault_max
        dip_max = 90 - theta_fault_min

        status_text = (f"Unstable Fault Orientations\n"
                       f"Min Dip: {dip_min:.1f}\u00B0\n"
                       f"Max Dip: {dip_max:.1f}\u00B0")
        text_color = 'red'

        # Draw Unstable Wedges in physical space (Conjugate pairs)
        w1 = patches.Wedge((0,0), 0.9, dip_min, dip_max, facecolor='red', alpha=0.4, zorder=1)
        w2 = patches.Wedge((0,0), 0.9, -dip_max, -dip_min, facecolor='red', alpha=0.4, zorder=1)
        w3 = patches.Wedge((0,0), 0.9, 180 - dip_max, 180 - dip_min, facecolor='red', alpha=0.4, zorder=1)
        w4 = patches.Wedge((0,0), 0.9, 180 + dip_min, 180 + dip_max, facecolor='red', alpha=0.4, zorder=1)
        ax2.add_patch(w1); ax2.add_patch(w2); ax2.add_patch(w3); ax2.add_patch(w4)

        # Calculate and draw "Optimal" fault plane based on Coulomb criterion
        opt_theta = 45 + np.degrees(np.arctan(mu))/2
        opt_dip = 90 - opt_theta
        ax2.plot([-0.9*np.cos(np.radians(opt_dip)), 0.9*np.cos(np.radians(opt_dip))],
                 [-0.9*np.sin(np.radians(opt_dip)), 0.9*np.sin(np.radians(opt_dip))], 'r--', linewidth=2, label='Optimal Plane')
        ax2.plot([-0.9*np.cos(np.radians(-opt_dip)), 0.9*np.cos(np.radians(-opt_dip))],
                 [-0.9*np.sin(np.radians(-opt_dip)), 0.9*np.sin(np.radians(-opt_dip))], 'r--', linewidth=2)
        ax2.legend(loc='lower center', bbox_to_anchor=(0.5, -0.1))

    else:
        ax1.set_title('Interactive Mohr Circle & Failure Envelope', color='black', fontsize=14)
        status_text = "Stable:\nNo slip on any plane."
        text_color = 'green'
        ax2.set_title("Physical Orientations\n(Stable)", color='green', fontsize=14)

    # Add text box for dynamic angle readouts to ax1
    ax1.text(0.02, 0.88, status_text, transform=ax1.transAxes, fontsize=12, color=text_color,
             bbox=dict(facecolor='white', edgecolor='black', alpha=0.9))

    # Formatting ax1
    ax1.set_xlim(0, 300)
    ax1.set_ylim(0, 200)
    ax1.set_xlabel(r'Normal Stress ($\sigma_n$) [MPa]', fontsize=12)
    ax1.set_ylabel(r'Shear Stress ($\tau$) [MPa]', fontsize=12)
    ax1.grid(True, linestyle='--', alpha=0.6)
    ax1.legend(loc='upper right')

    plt.tight_layout()
    plt.show()

# Create interactive UI using ipywidgets
ui = widgets.interact(plot_mohr_and_physical,
                 s1=widgets.FloatSlider(value=180, min=50, max=300, step=1, description='$\sigma_1$ (MPa)', continuous_update=False),
                 s3=widgets.FloatSlider(value=80, min=0, max=200, step=1, description='$\sigma_3$ (MPa)', continuous_update=False),
                 pf=widgets.FloatSlider(value=30, min=0, max=150, step=1, description='$P_f$ (MPa)', continuous_update=False),
                 mu=widgets.FloatSlider(value=0.6, min=0.1, max=1.0, step=0.05, description='Friction ($\mu$)', continuous_update=False))

<>:145: SyntaxWarning: invalid escape sequence '\s'
<>:146: SyntaxWarning: invalid escape sequence '\s'
<>:148: SyntaxWarning: invalid escape sequence '\m'
<>:145: SyntaxWarning: invalid escape sequence '\s'
<>:146: SyntaxWarning: invalid escape sequence '\s'
<>:148: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipython-input-2087687711.py:145: SyntaxWarning: invalid escape sequence '\s'
  s1=widgets.FloatSlider(value=180, min=50, max=300, step=1, description='$\sigma_1$ (MPa)', continuous_update=False),
/tmp/ipython-input-2087687711.py:146: SyntaxWarning: invalid escape sequence '\s'
  s3=widgets.FloatSlider(value=80, min=0, max=200, step=1, description='$\sigma_3$ (MPa)', continuous_update=False),
/tmp/ipython-input-2087687711.py:148: SyntaxWarning: invalid escape sequence '\m'
  mu=widgets.FloatSlider(value=0.6, min=0.1, max=1.0, step=0.05, description='Friction ($\mu$)', continuous_update=False))


interactive(children=(FloatSlider(value=180.0, continuous_update=False, description='$\\sigma_1$ (MPa)', max=3…